# **Analizador léxico-sintáctico**

### some advices 

## **1.Analizador léxico**

## **2.Analizador Sinctactico**

In [ ]:
import re

class error(Exception):
    def __init__(self,mensaje:str):
        super().__init__(mensaje)
        

class AnalizadorSintactico:
    def __init__(self,pseudocodigo:str):
        self.error_actual='empty'
        self.pseudocodigo = pseudocodigo
        self.indice_actual = 0
        self.indice=0

    def obtener_siguiente_estructura(self):
        if self.indice_actual < len(self.pseudocodigo):
            estructura_actual = self.pseudocodigo[self.indice_actual]
            self.indice_actual += 1
            return estructura_actual
        else:
            return None
        
    def obtener_siguiente_cadena(self,estructura:str):
        cadenas=estructura.split(" ")
        if cadenas and self.indice < len(cadenas):
            estructura_actual = cadenas[self.indice]
            self.indice += 1
            return estructura_actual
        else:
            self.indice = 0
            return None
        
    def condicion(self,cadena:str):
        regex=re.compile('[a-zA-Z0-9_]+=([a-zA-Z0-9_]+)')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def instrucciones(self,cadena:str):
        regex=re.compile('(([a-zA-Z0-9_(\s)+]+)(=|<|>|<=|>=)([a-zA-Z0-9_(\s+)]+))+')
        if re.fullmatch(regex,cadena):
            return True
        else:
            return False
    
    def escritura(self,cadena:str):
        regex = re.compile('Escribir\s+([a-zA-Z0-9_]+,)*[a-zA-Z0-9_]+;')
        if re.fullmatch(regex,cadena):
            return True
        else:   
            return False
    
    def si(self):
        estructura_actual = self.obtener_siguiente_estructura()
        cadena_actual = self.obtener_siguiente_cadena(estructura_actual)
        if cadena_actual and cadena_actual == "Si":
            cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
            print(cadena_actual)
            if cadena_actual and self.condicion(cadena_actual):
                cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                if cadena_actual and cadena_actual == "Entonces":
                    cadena_actual=self.obtener_siguiente_cadena(estructura_actual)
                    print(cadena_actual)
                    if cadena_actual and (self.instrucciones(cadena_actual) or self.escritura(cadena_actual)):
                        pass
                    else:
                        print('Error, se esperaba una instrucción valida')
                        self.lanzar_excepcion("Error , se esperaba una instrucción valida")
                else:
                    print("Error , se esperaba obtener la palabra 'Entonces'")
                    self.lanzar_excepcion("Error , se esperaba obtener la palabra 'Entonces'")
            else:
                print('Error, se esperaba una condicion valida')
                self.lanzar_excepcion("Error , se esperaba obtener una condicion valida")
        else:
            pass
        
    def lanzar_excepcion(self,mensaje:str):
        """Esta función lanza una excepción al encontrar un error en la cadena de entrada

        :param mensaje: El mensaje que quiere ser mostrado al usuario
        :type mensaje: str
        :raises self.error: Esta excepción es lanzada para indicar que se encontró un error
        """
        self.error_actual=mensaje
        raise error(self.error_actual)

if __name__ == '__main__':
    # regex=re.compile('[a-zA-Z0-9_]+=([a-zA-Z0-9_]+)')
    # print(re.fullmatch(regex,'hola=1'))
    an1=AnalizadorSintactico(['Si A=1 Entonces Escribir a;'])
    an1.si()


In [ ]:
# regex=re.compile('(([a-zA-Z0-9_]+)(=|<|>|<=|>=)([a-zA-Z0-9_]+)\s+)+{1,}')
# regex=re.compile('(([a-zA-Z0-9_(\s)+]+)(=|<|>|<=|>=)([a-zA-Z0-9_(\s+)]+))+')
# re.fullmatch(regex,'a>3 a>2 a=3')

# regex=re.compile('((\s*)Escribir) ( ((\s[a-zA-Z0-9_]+);) |  ((\s[a-zA-Z0-9_]+);)+ )')
# re.fullmatch(regex,'Escribir a;')

# regex=re.compile('((\s*)Escribir)(\s[a-zA-Z0-9_]+);')
# re.fullmatch(regex,'Escribir ae;')

import re

regex = re.compile('Escribir\s+([a-zA-Z0-9_]+,)*[a-zA-Z0-9_]+;')
print(re.fullmatch(regex,'Escribir a,b,c;'))

## GUI

In [ ]:
import tkinter as tk
from tkinter import messagebox
from ttkbootstrap import Style
import ttkbootstrap as tb

class GUIApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Analizador léxico-sintáctico")
        self.geometry("700x500")

        self.style = Style(theme='vapor')
        
        self.titulo=tb.Label(text='Analizador léxico-sintáctico',font=('Verdana',18),bootstyle='light')
        self.titulo.pack(side=tb.TOP,pady=15)
        
        self.contenedor = tb.Labelframe(text='Pseudocódigo', padding=35)
        self.contenedor.pack(side=tb.TOP,padx=(20,20),pady=(0,20)) 

        self.text_input = tb.Text(master=self.contenedor, width=70,height=10)
        self.text_input.pack(pady=10)
        
            
        self.buttonbar = tb.Frame()
        self.buttonbar.pack()
        
        self.b1 = tb.Button(master=self.buttonbar,width=20, text='Revisar', bootstyle='light',command=self.revisar)
        self.b1.pack(side=tb.LEFT,padx=10)
        
        self.b2=tb.Button(master=self.buttonbar,width=20,text='Borrar',bootstyle='light',command=self.borrar)
        self.b2.pack(side=tb.LEFT,padx=10)

        self.barra=tb.Progressbar(bootstyle="success-striped",maximum=101,value=0)
        self.barra.pack(pady=10)
        
        self.status_label=tb.Label(text='Status : ',font=('Verdana',18),bootstyle='light')
        self.status_label.pack(pady=10)
        

    def revisar(self):
        self.barra.step(100)  # Set the step value to the maximum value (100)
        self.status_label.config(text="Status: Revisar button clicked")
        messagebox.showinfo(message='Se ha revisado su pseudocódigo¡¡', title="Completado")

    def borrar(self):
        self.status_label.config(text="Status: Borrar button clicked")

    def run(self):
        self.mainloop()

if __name__ == '__main__':
    app = GUIApp()
    app.run()